# 에이전트 대화 시뮬레이션(고객 응대 시나리오)

참고: https://wikidocs.net/267816

## 환경 변수

In [1]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

True

In [3]:
import os
import getpass


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "01-LangGraph-Agent-Simulation"
_set_env("OPENAI_API_KEY")

## 상태 정의

In [5]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

## 상담사 에이전트

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage


def counselor_agent(messages: list[AnyMessage]):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

    response = llm.invoke(
        [SystemMessage("당신은 항공사의 고객 지원 담당자입니다. 한국어로 답변하세요.")]
        + messages
    )

    return {"messages": [response]}

In [7]:
from langchain_core.messages import HumanMessage

counselor_agent([HumanMessage("안녕하세요?")])

{'messages': [AIMessage(content='안녕하세요! 항공사 고객 지원팀입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 33, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CO0pwWPJbgvZfzmtCKtjYgBUXex2Z', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--513085da-b40d-4a2a-a884-b4fcea39463e-0', usage_metadata={'input_tokens': 33, 'output_tokens': 19, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}